In [1]:
import os, sys
from PIL import Image
import random
import numpy as np
from pathlib import Path

In [2]:
#Change path here:
path = "/Users/robertwalter/Desktop/RCI.2/2_MLMI/6_datasets/MontgomerySet_Versions/Version9"
#Change Names of output folder and rebinarized testImage labels here:
folder_out = "/out_image_bin"
folder_testlabels = "/testMasks_bin"

def calc_singleConfusion(out_array,label_array):
    if out_array.size == label_array.size:
        
        #Confusion Matrix with [tp,fp,tn,fn]
        confMatrix = [0] *4
        
        #print(len(out_array))
        #print(len(out_array[0]))
        for i in range(len(out_array)):
            for j in range(len(out_array[0])):
                
                if (label_array[i][j] != 0 and label_array[i][j] != 1) or (out_array[i][j] != 0 and out_array[i][j] != 1):
                    print("Image not binary")
                    return confMatrix
                else:   
                    #Negatives
                    if label_array[i][j] == 0:
                        #True-Negatives
                        if out_array[i][j] == label_array[i][j]:
                            confMatrix[2] += 1
                        #False-Negative
                        else:
                            confMatrix[3] += 1
                    #Positives
                    elif label_array[i][j] == 1:
                        #True-Positives
                        if out_array[i][j] == label_array[i][j]:
                            confMatrix[0] += 1
                        #False-Positives
                        else:
                            confMatrix[1] += 1

    else:
        print("Image Dimensions dont fit!")
        
    return confMatrix
        
def prepImage(img_name):
    img_out = Image.open(path+folder_out+"/"+img_name)
    img_label = Image.open(path+folder_testlabels+"/"+img_name)
    
    #check patient condition
    if img_name.endswith("0.png"):
        condFlag = 0
    elif img_name.endswith("1.png"):
        condFlag = 1   
    else:
        print(img_name+" patient condition not clear")
    
    arr_out = np.array(img_out)
    arr_label = np.array(img_label)
    #print(arr_out)
    return arr_out, arr_label, condFlag

#[tp,fp,tn,fn]

def calcRates(confMatrix):
    #[TPR,TNR,FNR,ACC] = [Sensitivitiy,Specificity, Miss Rate, Accuracy]
    rates = [0]*4
    
    rates[0] = confMatrix[0]/(confMatrix[0]+confMatrix[3])
    rates[1] = confMatrix[2]/(confMatrix[1]+confMatrix[2])
    rates[2] = confMatrix[3]/(confMatrix[3]+confMatrix[0])
    rates[3] = (confMatrix[0]+confMatrix[2])/sum(confMatrix)
    
    formatted_rates = [ '%.3f' % elem for elem in rates ]
    return formatted_rates

def main():
    #Counter [all,normal,abnormal]
    counter = [0] *3
    #Confusion Matrix 2x2x[all,normal,abnormal]
    confMatrix_complete = np.zeros((4,3))

    for img in os.listdir(path+folder_out):

        if img.endswith(".png"): 

            my_file = Path(path+folder_testlabels+"/"+img)
            if my_file.is_file():
                counter[0] += 1
                arr_out, arr_label, condFlag = prepImage(img)
                confMatrix = calc_singleConfusion(arr_out, arr_label) 
                #print(confMatrix)
                #all
                confMatrix_complete[:,0] += confMatrix
                
                #normal
                if condFlag==0:
                    counter[1] +=1
                    confMatrix_complete[:,1] += confMatrix
                #abnormal
                else:
                    counter[2] +=1
                    confMatrix_complete[:,2] += confMatrix
                    
                #Averaging                
 
          
            else:
                print(img+" Mask does not exist")
                
    confMatrix_complete[:,0] = confMatrix_complete[:,0]/counter[0]
    confMatrix_complete[:,1] = confMatrix_complete[:,1]/counter[1]
    confMatrix_complete[:,2] = confMatrix_complete[:,2]/counter[2]
    
    np.set_printoptions(precision=3)
    print("Confusion Matrix [tp,fp,tn,fn] average over all images ({}): {}".format(counter[0],confMatrix_complete[:,0]))
    print("Confusion Matrix [tp,fp,tn,fn] average over normal images ({}): {}".format(counter[1],confMatrix_complete[:,1]))  
    print("Confusion Matrix [tp,fp,tn,fn] average over abnormal images ({}): {}".format(counter[2],confMatrix_complete[:,2]))
    
    print("\n")
    
    for i in range(0,3):
        rates = calcRates(confMatrix_complete[:,i])
        if i == 0:
            np.set_printoptions(precision=5)
            print("Rates [TPR,TNR,FNR,ACC] for average over all images ({}): {}".format(counter[0],rates))
        elif i == 1:
            np.set_printoptions(precision=5)
            print("Rates [TPR,TNR,FNR,ACC] for average over normal images ({}): {}".format(counter[0],rates))
        elif i==2:
            np.set_printoptions(precision=5)
            print("Rates [TPR,TNR,FNR,ACC] for average over abnormal images ({}): {}".format(counter[0],rates))
            
    print("[TPR,TNR,FNR,ACC] = [Sensitivitiy,Specificity, Miss Rate, Accuracy]")
                

In [ ]:
main()

In [91]:
#testing
arr1 = np.array([[1,0,0],[1,1,1],[1,0,1]])
arr2 = np.array([[1,0,0],[1,0,1],[1,0,1]])
ConfMat = calc_singleConfusion(arr1,arr2)
print(ConfMat)

[5, 0, 3, 1]


In [4]:
confMatrix_complete[:,0] = confMatrix_complete[:,0]/counter[0]
confMatrix_complete[:,0] = confMatrix_complete[:,0]/172.800
confMatrix_complete[:,1] = confMatrix_complete[:,1]/counter[1]
confMatrix_complete[:,1] = confMatrix_complete[:,1]/172.800
confMatrix_complete[:,2] = confMatrix_complete[:,2]/counter[2]
confMatrix_complete[:,2] = confMatrix_complete[:,2]/172.800


np.set_printoptions(precision=3)
print("Confusion Matrix [tp,fp,tn,fn] average over all images ({}): {}".format(counter[0],confMatrix_complete[:,0]))
print("Confusion Matrix [tp,fp,tn,fn] average over normal images ({}): {}".format(counter[1],confMatrix_complete[:,1]))  
print("Confusion Matrix [tp,fp,tn,fn] average over abnormal images ({}): {}".format(counter[2],confMatrix_complete[:,2]))


NameError: name 'confMatrix_complete' is not defined